In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `/central/groups/esm/jschmitt/experiments/feature_importance`


In [2]:
import EnsembleKalmanProcesses as EKP
import ScikitLearn
import YAML
import JLD2
import ClimaCalibrate as CAL
import CalibrateEmulateSample as CES
using Statistics
using CairoMakie

In [3]:
simdir = "/central/scratch/cchristo/edmf_impl_dev4/exp24"

# load objects 
eki = JLD2.load_object(simdir * "/iteration_005/" * "eki_file.jld2")
prior = CAL.get_prior(simdir * "/configs/" * "prior.toml");


In [4]:
u_ar = EKP.get_u(eki)
ϕ_ar = EKP.get_ϕ(prior, eki);

In [4]:
# extract training pairs from CES
training_pairs = CES.Utilities.get_training_points(eki, 5)

Y = training_pairs.inputs.stored_data
X = training_pairs.outputs.stored_data

450×1000 Matrix{Float64}:
 NaN  NaN  -0.786581  NaN  -0.789177  …  -0.787218  -0.774074  -0.775963
 NaN  NaN  -0.784856  NaN  -0.782471     -0.785951  -0.77053   -0.773113
 NaN  NaN  -0.782901  NaN  -0.779809     -0.78478   -0.767738  -0.770851
 NaN  NaN  -0.777458  NaN  -0.770114     -0.78322   -0.763391  -0.76581
 NaN  NaN  -0.754195  NaN  -0.737448     -0.77879   -0.750477  -0.746282
 NaN  NaN  -0.701632  NaN  -0.681481  …  -0.762366  -0.717259  -0.711948
 NaN  NaN  -0.643697  NaN  -0.603201     -0.721708  -0.662628  -0.657243
 NaN  NaN  -0.566422  NaN  -0.564412     -0.637772  -0.59453   -0.597246
 NaN  NaN  -0.509719  NaN  -0.470425     -0.504008  -0.507891  -0.535852
 NaN  NaN  -0.388017  NaN  -0.332571     -0.365707  -0.390785  -0.409201
   ⋮                                  ⋱                        
 NaN  NaN  -0.765171  NaN   1.42862       1.51535    1.54458    1.57666
 NaN  NaN   0.9267    NaN   1.45256       1.53243    1.51677    1.57231
 NaN  NaN   1.32629   NaN   1.47387  

In [22]:
any(isnan.(X[:, 3]))

false

In [23]:
lines(X[:, 3])

UndefVarError: UndefVarError: `lines` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
using DecisionTree
model = RandomForestRegressor(n_trees = 100, max_depth = 5)
fit!(model, X', Y[1, :]) # fit the opposite way 

In [32]:
# load toml 
using TOML 
perturbations = [0.8, 1.0, 1.2]
default_toml = TOML.parsefile("tomls/simple_precalibrated.toml")
toml_list = []
exp_root = "perturbed"
if !isdir(exp_root)
    mkdir(exp_root)
end
# default_toml[collect(keys(default_toml))[1]]["value"]
for (k, v) in default_toml
    println(k)
    if !isdir(joinpath(exp_root, k))
        mkdir(joinpath(exp_root, k))
    end
    #println(v["value"])
    for perturbation in perturbations
        cptoml = deepcopy(default_toml)
        println("perturbation: ", perturbation)
        cptoml[k]["value"] = v["value"] .* perturbation
        println(cptoml[k]["value"])
        if !isdir(joinpath(exp_root, k, string(perturbation)))
            mkdir(joinpath(exp_root, k, string(perturbation)))
        end
        # save new cptoml to this directory
        open(joinpath(exp_root, k, string(perturbation), "parameters.toml"), "w") do io
            TOML.print(io, cptoml)
        end
        push!(toml_list, joinpath(exp_root, k, string(perturbation), "parameters.toml"))
    end
    println(" ")
end



mixing_length_tke_surf_scale
perturbation: 0.8
0.4405975125925878
perturbation: 1.0
0.5507468907407347
perturbation: 1.2
0.6608962688888816
 
mixing_length_diss_coeff
perturbation: 0.8
0.10812017509593222
perturbation: 1.0
0.13515021886991527
perturbation: 1.2
0.16218026264389832
 
diagnostic_covariance_coeff
perturbation: 0.8
1.4869700341824448
perturbation: 1.0
1.858712542728056
perturbation: 1.2
2.230455051273667
 
entr_param_vec
perturbation: 0.8
[0.6585961475698758, 0.0470215276565385, 0.3050301571066385, 1.4017120407046655, -0.25719862728188864, -0.015811905221975134, -0.0474151148915126, 0.2888208640927998, 0.4328989052513739, -0.3745204090910681, 0.8539956129081535, 0.31035694755347204]
perturbation: 1.0
[0.8232451844623447, 0.058776909570673115, 0.3812876963832981, 1.7521400508808318, -0.32149828410236075, -0.019764881527468917, -0.05926889361439075, 0.3610260801159997, 0.5411236315642174, -0.4681505113638351, 1.0674945161351919, 0.38794618444184004]
perturbation: 1.2
[0.98789

In [33]:
toml_list

33-element Vector{Any}:
 "perturbed/mixing_length_tke_surf_scale/0.8/parameters.toml"
 "perturbed/mixing_length_tke_surf_scale/1.0/parameters.toml"
 "perturbed/mixing_length_tke_surf_scale/1.2/parameters.toml"
 "perturbed/mixing_length_diss_coeff/0.8/parameters.toml"
 "perturbed/mixing_length_diss_coeff/1.0/parameters.toml"
 "perturbed/mixing_length_diss_coeff/1.2/parameters.toml"
 "perturbed/diagnostic_covariance_coeff/0.8/parameters.toml"
 "perturbed/diagnostic_covariance_coeff/1.0/parameters.toml"
 "perturbed/diagnostic_covariance_coeff/1.2/parameters.toml"
 "perturbed/entr_param_vec/0.8/parameters.toml"
 ⋮
 "perturbed/mixing_length_Prandtl_number_0/0.8/parameters.toml"
 "perturbed/mixing_length_Prandtl_number_0/1.0/parameters.toml"
 "perturbed/mixing_length_Prandtl_number_0/1.2/parameters.toml"
 "perturbed/mixing_length_static_stab_coeff/0.8/parameters.toml"
 "perturbed/mixing_length_static_stab_coeff/1.0/parameters.toml"
 "perturbed/mixing_length_static_stab_coeff/1.2/parameters.t

In [ ]:
for param in params 
    for percent in [.8, .9, 1. ,1.1, 1.2]
        TOML = toml.copy()
        TOML["prior"]["parameters"][param]["mean"] = percent * prior.parameters[param].mean
        TOML.print("parameter/percent.toml")
    end
end

# save toml to each member folder
CAL.run_worker_iteration()